# LSTM Model

This notebook implements the LSTM model for time series forecasting of household electricity consumption.

In [ ]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

data = pd.read_csv('../data/preprocessed_household_power_consumption.csv', index_col='datetime', parse_dates=True)

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data[['Global_active_power']])

train_data = scaled_data[:-30]
test_data = scaled_data[-30:]

def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X[i:(i + time_steps)]
        Xs.append(v)
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

TIME_STEPS = 10
X_train, y_train = create_dataset(train_data, train_data, TIME_STEPS)
X_test, y_test = create_dataset(test_data, test_data, TIME_STEPS)

model = Sequential()
model.add(LSTM(units=50, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    shuffle=False
)

y_pred = model.predict(X_test)
y_pred_inverse = scaler.inverse_transform(y_pred)

plt.figure(figsize=(14, 7))
plt.plot(data.index[-30:], data['Global_active_power'].iloc[-30:], label='True')
plt.plot(data.index[-30:], y_pred_inverse, label='LSTM Forecast')
plt.xlabel('Datetime')
plt.ylabel('Global Active Power')
plt.title('LSTM Model Forecast')
plt.legend()
plt.show()

lstm_rmse = np.sqrt(mean_squared_error(data['Global_active_power'].iloc[-30:], y_pred_inverse))
print(f'LSTM RMSE: {lstm_rmse}')